# Tanzanian Water Well Classification Analysis

#### By Jonny Hofmeister

This data comes from Taarifa, an open source platform that tracks infrastucture related issues and data, who has sourced this data from the Tanzanian Ministry of Water. It is posted on the [DRIVENDATA](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/) site as a classification data science contest.

###### This project will utilize the CRISP-DM process.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from category_encoders import CountEncoder

from functions import metrics_matrix

In [3]:
# import data
dftarget = pd.read_csv('data/train_target.csv')
dfvalues = pd.read_csv('data/train_values.csv')

# merge on id
df = pd.merge(dftarget, dfvalues, how='inner', on='id')

## Business Understanding

The goal of this classification model is to predict the functional status of water well pumps in Tanzania. We are supplied with ample data for well pumps with known functionality, but there are many others out there whose functionality is unknown. We will try to mine this data for insights that can help the Tanzanian Ministry of Water predict if wells with unknown functionality are working or not. This information will help them to understand and determine the best practices for addressing repair of wells throughout the country. 

The first goal of this project is to supply a prediction model with ample accuracy so that the Tanzanian government and associated groups can analyze their next steps with a more complete understanding of well functionality accross the country. 

After building a successful prediction model, the next steps are to help the Tanzanian Ministry of Water determine the most efficient ways to go about repairing wells that need repair. Given the remote areas and limited repair labor, they must determine how to prioritize repair in areas and wells that would maximize the utility of their mission. 

## Data Understanding

### Target

The classification target of well status given in the data is grouped into three classes: 'functional', 'non-functional', and 'functional needs repair'. This classification problem could reduced to binary targets (ones needing repair or not), but given that TMW might want to prioritize completely non-functional wells over functional-needs-repair ones, we will use a ternary classification model. 

Target classes and frequencies are shown below.

In [4]:
df['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

#### Evaluation Metrics

We also must address the metric on which we will measure the success of the classification model. Given different contexts for problems, one may prefer to maximize precision or recall, or perhaps order and ROC-AUC score. In this problem, I dont believe either a false positive or a false negative is worse than the other. Identifying a non-working well as working means that the well would be missed by the TSM and repair teams - this isn't good. But, identifying a working well as not-working means you may end up sending a repair person to a working well and this could mean that there isn't time to go find and fix a well that is truly broken (there are thousands of wells so we must assume the TSM will not get to every well that needs repair). In both FP/FN cases it is possible that a well that needs repair doesn't get it, and for this reason, this model will focus on maximizing the F1 score, which balances both precision and recall. 


### Features

There are around 40 features given in the original dataset. They contain both numerical and categorical information. Many of the features are repetitive though, for example, 'extraction_type', 'extraction_type_class', and 'extraction_type_group' are almost identical. Each contains multiple of the same classes but 'class' or 'group' may have an extra couple labels created in place of a previous label. Because all of these reduntant features, we must decide which ones to use in modeling to avoid colinearities. 

In [5]:
# the 'duplicate' columns can easily be seen in a list
list(df.columns)

['id',
 'status_group',
 'amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

I have explored and selected this list below, which removes 'duplicate' type columns but still keeps one from each group. I would consider this data to encompass the whole dataset - just without colinearities. Refining further and removing columns without much variance is an option I may explore after creating some models.

In [6]:
keep_columns = ['status_group', 'amount_tsh', 'gps_height', 'num_private', 'region', 'district_code', 'lga',
                'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type',
                'management_group', 'water_quality', 'quantity', 'source', 'waterpoint_type', 
                'population']

df = df[keep_columns]

### Description of Columns
Here are brief descriptions of the columns selected.
- 'amount_tsh': Total Static Head - the height of the water column produced by the pressure of the pump.
- 'gps_height': The altitude of the pump.
- 'num_private': No description given - possibly number of private entities (companies groups), using this well.
- 'region': The geographical region the pump is located in.
- 'district_code': The district (coded) pump is located in.
- 'lga': Another geographical region description (more specific than the two above).
- 'public_meeting': Boolean - if there has been a public meeting where this well has been addressed.
- 'scheme_management': Group who operates the well.
- 'permit': Boolean - if the well has a permit or not.
- 'construction_year': The year the well was created.
- 'extraction_type': The type of water extraction the well uses.
- 'management_group': Like scheme management, but if that group is commercial or user-based.
- 'water_quality': The quality of water at that well.
- 'quantity': How much water is available to the pump - categorical.
- 'source': The water source type for the waterpoint (groundwater, rainwater, etc).
- 'waterpoint_type': Type of waterpoint access (handpump, spring, etc).
- 'population': The population near the well.

## Data Preparation

#### Missing Values

I have chosen the columns above over their counterparts for certain reasons, the main one being missing values. These columns contain less NaNs or missing groups than the others, so using these will give us access to more of our data in training than the other. Note: other columns may contain their own 'unknown' group, but only these contain actual NaN values that will throw errors in our modeling and must be grouped and labeled or removed. 

In [7]:
df[['public_meeting', 'scheme_management', 'permit']].isna().describe()

,public_meeting,scheme_management,permit
count,59400,59400,59400
unique,2,2,2
top,False,False,False
freq,56066,55523,56344


Of the remaining columns only 3 contain NaNs - public_meeting, permit, and scheme_management - and around 3-4 thousand missing values for each. 

The missing values in scheme_management can be easily imputed in preprocessing, so these will be left in order to preserve more rows of data. The public_meeting and permit columns are a bit trickier to deal with since they are boolean True/False values. If we wanted to impute and group the NaNs into an 'unknown' category, the model could no longer view it as a single column of 1s and 0s - we would be forced to one hot encode and turn 1 column to 3, or to frequency encode. 

In [8]:
df.dropna(subset=['public_meeting', 'permit'], inplace=True)

I have decided to drop NaNs from the boolean columns but retain the others and count encode the NaNs into an 'unknown' column.

#### Data Types

The last main part of data preparation is to deal with different data types in the features. We have a few numerical columns that must be imputed for missing values and scaled. The majority of the columns are categorical data -  these can be dealt with in multiple ways. One Hot Encoding is a good option, but not for a dozen columns with dozens of values. The best way to handle this is to select important colummns that have fewer numbers of values to One Hot Encode, and then trasnform the others with some kind of Frequency or Count encoder. Target Encoding or LeaveOneOut encoding is another successful way to encode how frequency of a value relates to the probability of a target class. These forms of encoding all remain one column - making it a good way to include categorical columns without creating more features. 

Let's start by looping through the columns to extract different data types. We will set a threshold value for the number of categories in a categorical column to decide if it will be One Hot Encoded or frequency encoded. 

In [9]:
# First split the data into features and target
X = df.drop('status_group', axis=1)
y = df['status_group']

# We also need to first change the datatype of the district code column
# It is numeric, but we want to treat it as categorical
X['district_code'] = X['district_code'].astype('object')

# Create arrays for different column types
num_cols = []
ohe_cols = []
freq_cols = []

# Set a threshold value for the maximum number of unqiue values we want to OHE
thresh = 7


# Loop through the columns and append col to proper list
for col in X.columns:
    
    if X[col].dtype in ['float64', 'int64']:
        num_cols.append(col)
        continue
    
    elif X[col].dtype == 'object':
        if len(X[col].unique()) > thresh:
            freq_cols.append(col)
        elif len(X[col].unique()) <= thresh:
            ohe_cols.append(col)

But wait! Remeber two of our colunmns are boolean and contain only True/False. These will end up in the ohe_cols list but we dont want to include them there. Quickly remove them and create a separate list for T/F cols.

In [10]:
# The permit and public meeting columns always end up at the top of the list, so can easily select them as below

TF_cols = ohe_cols[0:2]
ohe_cols = ohe_cols[2:]
TF_cols

['public_meeting', 'permit']

## Preprocessing

In order to smoothly process all the columns at once, we will define some processing steps into piplines and then apply those to a column transformer which will easily manipulate our training data. Because we don't want any "leakage" while scaling and processing the data, the very important Train / Test split will go right here.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

#### Numeric Columns

First I will define the processing steps for the numeric columns. But we may want to treat those numerics slightly differently so lets consider some things first. 

The construction year column contains a decent amount of zero columns - we will deal with these by imputing them with the median year. We dont want to replace zeros in the other columns with their median though,because we cannot be sure if the zero is the actual value or a NaN placeholder like it is for a construction year. Because year is ordinal/interval, I will scale it using a Min/Max scaler. 

The num_cols list created above will have to be broken up into more specific list to deal with the difference in numeric types.

In [12]:
year_col = ['construction_year']

year_scaler = Pipeline(steps=[
    ('year_imputer', SimpleImputer(missing_values=0, strategy='median')),
    ('year_scaler', MinMaxScaler())])

The rest of the numeric columns will use not impute any values and use a Min Max scaler, and we dont have any NaNs to fill, so we dont need a pipeline here, we can just call the scaler we want in the column transformers. But we still need a list of those columns:

In [13]:
num_cols = ['amount_tsh', 'gps_height', 'num_private', 'population']

#### Frequency Encoding Columns

The steps to process frequency encoded columns are simple. Impute the NaNs (only contained in scheme_management currently) into an 'unknown' group, and then transform using a normalized count encoder. 

In [14]:
freq_encoder = Pipeline(steps=[
    ('freq_imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('freq_enc', CountEncoder(normalize=True, min_group_size=0.01, min_group_name='unknown'))
])

#### One Hot Columns

One hot encoding is fairly straight forward and could be one step. But, because we may change the threshold for one hot ecoding and could encounter the scheme management column with NaNs, I am going to add a simple imputer step to deal with it. This will not effect any of the other columns.

In [15]:
ohe_transformer = Pipeline(steps=[
    ('ohe_imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('ohencoder', OneHotEncoder(handle_unknown='ignore'))])

#### Boolean Columns

Since our True/False columns already contain no NaNs and can be hanlded by sklearn models, we dont need to define any steps to preprocess them. In our column transformer, they will be left out of the columns list, but setting remainder to 'passthrough' will make sure they move along into the processed dataframe untouched. 

#### Preprocessor

We can now use Sklearns Column Transformer to easily perform all these steps on their respective columns at once.

In [16]:
preprocessor = ColumnTransformer(transformers=[
    ('year', year_scaler, year_col),
    ('num', MinMaxScaler(), num_cols),
    ('freq', freq_encoder, freq_cols),
    ('ohe', ohe_transformer, ohe_cols)],
    remainder='passthrough')

#  Modeling

### Baseline
In order to evaluate the success of following models, we create a baseline to compare to. The most naive baseline is to predict the majority class. Here, if we predicted every well to be functional, we would be correct about 54% of the time. But this baseline model is useless in practice -- if we assumed all are functional then how would we go about repairing the ones that are non-functional?

We can create a better simple baseline model that predicts 2 classes and could be actionable by the TMW to some degree. Let's take the column that measures total static head of the pump ('amount_tsh') and wherever it is zero, predict 'non-functional', and wherever it is greater than zero, predict 'functional'. We can compute our success rate for this model below:

In [17]:
# Number correct is number of functional pumps with tsh > 0 and non-functional pumps with tsh=0.
func_correct = len(df[(df['amount_tsh']>0) & (df['status_group']=='functional')])
nonfunc_correct = len(df[(df['amount_tsh']==0) & (df['status_group']=='non functional')])

# Add totals
total_correct = func_correct + nonfunc_correct
print(f'Total Static Head guess baseline accuracy: {total_correct/len(df):.2}%')

Total Static Head guess baseline accuracy: 0.53%


This may be a lower accuaracy than the 54% majority class, but is at least actionable in terms of understanding if pumps are working or not. If they decided to go fix every pump labeled non-functional in this model, slightly over half of these pumps would be broken or in need of repair. We would have guessed 36,428 as non-functional, 16,555 would acutally be non-functional, and a lucky 2637 would be functional in need of repair. This adds up to a repair efficiency of 52.6%, which happens to be right on par with our overall prediction accuracy for this model. 

But, this doesn't take in to account the 3794 non-functional pumps with a TSH greater than zero we mis-labeled. While this number is smaller than expected, we would still like to be more accurate in order to identify all broken pumps and to make sure we dont waste any time going to fix functional ones (maximising both precision and recall). 

For now, if a model is worthwhile or not will be determined by if we can beat that 53% accuracy above. 

## First Model: K-Nearest Neighbors

The first model we will explore is K-Nearest Neighbors. Computing distance and similarity between rows seems like it could yeild some useful results. Unfortunately this will also require quite a lot of time considering both the number of rows and columns we have. If no others perform well and this does, it may be worth trying to reduce the number of columns in order to perform a grid search and improve the KNN.

Because we have predefined our processing steps, we can call a simple pipeline that processes and then fits the data to a Sklearn KNN model.

In [18]:
from sklearn.neighbors import KNeighborsClassifier

In [19]:
knn_clf = Pipeline(steps=[('preprocessor', preprocessor),
                         ('knn', KNeighborsClassifier())])

knn_clf.fit(X_train,y_train);

## Model 2: Random Forest

I think a random forest will be a good way to mine insights from this data. Thinking back to the baseline model we created, a decision tree works in the same way. Only using one if statement - if amount_tsh > 0, predict functional - we were able to achieve 53% accuracy. Following this same logic, perhaps a few more layers of decision based sorting can lead to sucessful predictions. Another benefit is it is slightly faster than KNN, making a grid search less daunting.

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
forest_clf = Pipeline(steps=[('preprocessor', preprocessor),
                         ('classifier', RandomForestClassifier(random_state=1234))])

forest_clf.fit(X_train, y_train);

In [20]:
forest_clf = Pipeline(steps=[('preprocessor', preprocessor),
                         ('classifier', RandomForestClassifier(random_state=1234,
                                                              n_estimators=160,
                                                              min_samples_split=2,
                                                              min_samples_leaf=2,
                                                              max_depth=60))])

forest_clf.fit(X_train, y_train);

## Model 3: Support Vector Classifier

The third model we will test is a SVC. The reason for trying a SVC is to a grid search with SVCs different kernel options. This machine learning algorithm manipulates feature space in ways that the previous two models cannot. In my opinion it is worth searching these to see if one is promising enough to continue with.

In [22]:
from sklearn.svm import SVC

In [23]:
svc_clf = Pipeline(steps=[('preprocessor', preprocessor),
                         ('classifier', SVC())])

In [24]:
param_grid = {
    'classifier__kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

SVC_GS = GridSearchCV(svc_clf, param_grid, cv=3, scoring='f1_weighted')
SVC_GS.fit(X_train, y_train);

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## Evaluation
